### Формирвание сводных таблиц для текстового комментария
Скрипт парсит данные с многостраничного частично стандартизированного отчета, сводит их в единую своднуб таблицу и сохраняет в необходимом заказчику формате.

In [1]:
import openpyxl as opx
from openpyxl.styles import numbers
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

In [ ]:
# СКРИПТ ЧАСТИЧНО ОТРЕДАКТИРОВАН ТАК КАК СОДЕРЖИТ ДАННЫЕ КЛИЕНТА

# ЗАГРУЗКА В ОПЕРАТИВКУ ФАЙЛА ВЫГРУЗКИ ДЛЯ ЧТЕНИЯ

wb_path = f'Башкортостан 2021.04_Done.xlsx'
workbook = opx.load_workbook(wb_path, read_only=True)
workbook.sheetnames

In [3]:
# ВХОДНЫЕ ДАННЫЕ

# КОЛОНКА ОБРАБАТЫВАЕМОГО МЕСЯЦА
# НАПРИМЕР, AF - Июнь 2020
col = 'AP'

s = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
columns = [i for i in s] + [f'{s[i]}{l}' for i in range(2) for l in s]

lastYear_col = columns[columns.index(col)-12]

currentMonth = workbook['Все сайты холдинга'][f'{col}3'].value
lastYearMonth = workbook['Все сайты холдинга'][f'{lastYear_col}3'].value

export_name = f'Башкирия. Сводные таблицы {currentMonth}.xlsx'

currentMonth, lastYearMonth

('апр.21', 'апр.20')

In [4]:
big_dict = {
    '1':             {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '1._Баш.трафик':  {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '1._Не-Баш.траф': {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '2':               {'users': 5,  'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '2_Баш.трафик':     {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '2_Не-Баш.трафик':  {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '3':                   {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '3_Баш.трафик':         {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '3_Не-Баш.трафик':      {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '4':                {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '4_Баш.трафик':      {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '4_Не-Баш.трафик':   {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '5':                 {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '5_Баш.трафик':       {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '5_Не-Баш.трафик':    {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '6':                  {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '6_Баш.трафик':        {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '6_Не-Баш.трафик':     {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '7':               {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '7_Баш.трафик':     {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '7_Не-Баш.трафик':  {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '8':               {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '8_Баш.трафик':     {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '8_Не-Баш.трафик':  {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27},
    '9':             {'users': 5, 'visits': 6, 'views': 7, 'yanews': 37, 'zen': 39, 'google': 43, 'local users': 15},
    '9._Баш.трафик':  {'users': 5, 'visits': 6, 'views': 7, 'yanews': 29, 'zen': 31, 'google': 35},
    '9._Не-Баш.траф': {'users': 5, 'visits': 6, 'views': 7, 'yanews': 21, 'zen': 23, 'google': 27}
}

In [5]:
# ПРОВЕРЯЮ ДЛЯ КАЖДОГО ЛИСТА, что на заданных строках в big_dict находятся именно УНИКАЛЫ и СЕССИИ
# Если НЕ Посетители и Сессии, строка выйдет на печать
for _ in big_dict.keys():
    users = workbook[_][f"A{big_dict[_]['users']}"].value
    visits = workbook[_][f"A{big_dict[_]['visits']}"].value
    if users!='Посетители' or visits!='Сессии':
        print(f"{users}\t{visits}\t{_}")

In [6]:
# ПРОВЕРЯЮ ДЛЯ КАЖЖДОГО ЛИСТА, что на заданных строках в big_dict 
# находятся именно Яндекс.Новости, Zen.Yandex, Google Discover
# Если НЕ Яндекс.Новости, Zen.Yandex, Google Discover соответственно, строка выйдет на печать

for _ in big_dict.keys():
    title = workbook[_]["A1"].value
    yanews = workbook[_][f"A{big_dict[_]['yanews']-1}"].value
    zen = workbook[_][f"A{big_dict[_]['zen']-1}"].value
    google = workbook[_][f"A{big_dict[_]['google']-1}"].value
    if google!='Google Discover' or zen!='Zen.yandex' or yanews!='Яндекс.Новости':
        print(f"{yanews}\t{zen}\t{google}\t{_}\t\t{title}")

In [ ]:
data_sheets = [_ for _ in big_dict.keys()]
title = [workbook[_][f"A1"].value for _ in big_dict.keys()]

users = [workbook[_][f"{col}{big_dict[_]['users']}"].value for _ in big_dict.keys()]
users_lastYear = [workbook[_][f"{lastYear_col}{big_dict[_]['users']}"].value for _ in big_dict.keys()]

visits = [workbook[_][f"{col}{big_dict[_]['visits']}"].value for _ in big_dict.keys()]
visits_lastYear = [workbook[_][f"{lastYear_col}{big_dict[_]['visits']}"].value for _ in big_dict.keys()]

views = [workbook[_][f"{col}{big_dict[_]['views']}"].value for _ in big_dict.keys()]
views_lastYear = [workbook[_][f"{lastYear_col}{big_dict[_]['views']}"].value for _ in big_dict.keys()]

yanews = [workbook[_][f"{col}{big_dict[_]['yanews']}"].value for _ in big_dict.keys()]
yanews_lastYear = [workbook[_][f"{lastYear_col}{big_dict[_]['yanews']}"].value for _ in big_dict.keys()]

zen = [workbook[_][f"{col}{big_dict[_]['zen']}"].value for _ in big_dict.keys()]
zen_lastYear = [workbook[_][f"{lastYear_col}{big_dict[_]['zen']}"].value for _ in big_dict.keys()]

google = [workbook[_][f"{col}{big_dict[_]['google']}"].value for _ in big_dict.keys()]
google_lastYear = [workbook[_][f"{lastYear_col}{big_dict[_]['google']}"].value for _ in big_dict.keys()]

data = pd.DataFrame({#'Листы': data_sheets, 
                     f'Уникалы {currentMonth}': users,
                     f'Уникалы {lastYearMonth}': users_lastYear,
                     f'Сессии {currentMonth}': visits,
                     f'Сессии {lastYearMonth}': visits_lastYear,
                     f'Просмотры {currentMonth}': views,
                     f'Просмотры {lastYearMonth}': views_lastYear,    
                     f'Яндекс.Новости {currentMonth}': yanews,
                     f'Яндекс.Новости {lastYearMonth}': yanews_lastYear,
                     f'Zen.yandex {currentMonth}': zen,
                     f'Zen.yandex {lastYearMonth}': zen_lastYear,
                     f'Google Discover {currentMonth}': google,
                     f'Google Discover {lastYearMonth}': google_lastYear,
                     f'Заголовок листа. А1': title},
                   index=data_sheets)

data.head()

In [ ]:
data['Уникалы YoY'] = data[f'Уникалы {currentMonth}']/data[ f'Уникалы {lastYearMonth}']-1
data['Сессии YoY'] = data[f'Сессии {currentMonth}']/data[ f'Сессии {lastYearMonth}']-1
data['Просмотры YoY'] = data[f'Просмотры {currentMonth}']/data[ f'Просмотры {lastYearMonth}']-1
data['Яндекс.Новости YoY'] = data[f'Яндекс.Новости {currentMonth}']/data[ f'Яндекс.Новости {lastYearMonth}']-1
data['Zen.yandex YoY'] = data[f'Zen.yandex {currentMonth}']/data[ f'Zen.yandex {lastYearMonth}']-1
data['Google Discover YoY'] = data[f'Google Discover {currentMonth}']/data[ f'Google Discover {lastYearMonth}']-1

data = data[[
    f'Уникалы {lastYearMonth}', f'Уникалы {currentMonth}', 'Уникалы YoY',
    f'Сессии {lastYearMonth}', f'Сессии {currentMonth}', 'Сессии YoY',
    f'Просмотры {lastYearMonth}', f'Просмотры {currentMonth}', 'Просмотры YoY',    
    f'Яндекс.Новости {lastYearMonth}', f'Яндекс.Новости {currentMonth}', 'Яндекс.Новости YoY',
    f'Zen.yandex {lastYearMonth}', f'Zen.yandex {currentMonth}', 'Zen.yandex YoY',
    f'Google Discover {lastYearMonth}', f'Google Discover {currentMonth}', 'Google Discover YoY',
    'Заголовок листа. А1'
            ]]
data.head()

In [16]:
# ЧАСТЬ СКРИПТА ВЫРЕЗАНА, ТАК КАК СОДЕРЖИТ ДАННЫЕ КЛИЕНТА

# ЗАПИСЬ ДАТАФРЕЙМОВ В ЭКСЕЛЬ, ФОРМАТИРОВАНИЕ И СОХРАНЕНИЕ В ФАЙЛ

wb = opx.Workbook()

ws_data = wb.create_sheet("Все данные")
ws_common_data = wb.create_sheet("С вкладок Общий")
ws_bash_data = wb.create_sheet("С вкладок Баш.траф")
ws_nobash_data = wb.create_sheet("С вкладок НЕ-Баш.траф")
ws_local = wb.create_sheet("Рег.аудитория всех изданий")
del wb['Sheet']

ws_list = [ws_data, ws_common_data, ws_bash_data, ws_nobash_data, ws_local]
df_list = [data, common_data, bash_data, nobash_data, local]

for i, df in enumerate(df_list):
    ws = ws_list[i]
    
    #     SET THE COLUMNS WIDTH
    ws.column_dimensions['A'].width = 25
    for c in columns[1:]:
         ws.column_dimensions[c].width = 15
    
    #     WRITE DATAFRAMES TO DIFFERENT SHEETS
    for r in dataframe_to_rows(df, index=True, header=True):
        ws.append(r)
    
    #     FORMATTING THE SHEET
    ws.row_dimensions[1].height = 55
    ws.delete_rows(2)
    for row in ws.rows:
        for cell in row:
            cell.font = font = opx.styles.Font(name='Arial', size=10)
    for cell in ws['1']:
        cell.font = opx.styles.Font(name='Arial', size=10, bold=True)
        cell.alignment = opx.styles.Alignment(wrapText = True)
    for cell in ws['A']:
        cell.font = opx.styles.Font(name='Arial', size=10, bold=True)
        
    #     SET THE AUTO-FILTER
    c = columns[ws.max_column-1] 
    ws.auto_filter.ref = f"A1:{c}{ws_data.max_row}"
    
    #     FREEZE FIRST COLUMN A
    ws.freeze_panes = 'B1'

# FORMATTING THE NUMBERS 
for c in columns[1:18:3] + columns[2:18:3]:
    for cell in ws_data[c][1:]:
        cell.number_format = '# ### ###'
for c in columns[3:19:3]:
    for cell in ws_data[c][1:]:
        cell.number_format = numbers.FORMAT_PERCENTAGE          

for sheet in ws_list[1:]:
    for c in ['B', 'D', 'F', 'H', 'J']:
        for cell in sheet[c][1:]:
            cell.number_format = '# ### ###'    
    for c in ['C', 'E', 'G', 'I', 'K']:
        for cell in sheet[c][1:]:
            cell.number_format = numbers.FORMAT_PERCENTAGE
        
wb.save(export_name)